## **Min total completion time by single machine**

 ***MIP Model***

***Indices and Parameters***

- $J = \{1, 2, ..., n\}$: A set of machines
- $p_{j}$: processing time of job $j$.


***Decision Variables***

- $x_j$: completion time of job $j$
- $z_{i,j}$: Binary variable that equals 1 if $j$ ---> $i$. otherwise 0

**Objective Function:**


$$ \text{Minimize} \quad Z = \sum_{j \in J} x_j $$

**Constraints:**

1. $$ x_j + p_i - x_i \leq M \cdot z_{ij}, \quad \forall i, j \in J, i \neq j $$
2. $$ x_i + p_j - x_j \leq M \cdot (1 - z_{ij}), \quad \forall i, j \in J, i \neq j $$
3. $$ x_j \geq p_j, \quad \forall j \in J $$

***Code***

In [5]:
import pulp

class SingleMachineTCT:
    M = 10000
    def __init__(self, I, J, p):
        self.J = J
        self.I = I
        self.p = p
        self.M = 10000

        #initialize the problem
        self.problem = pulp.LpProblem("SingleMachineTCT", pulp.LpMinimize)
        
        #DVs
        self.x = pulp.LpVariable.dicts("x", J, lowBound = 0 , cat = pulp.LpContinuous)
        self.z = pulp.LpVariable.dicts("z", [(i,j) for i in self.I for j in self.J if i!=j], cat = pulp.LpBinary)
    
    def model(self):
        #obj
        self.problem += pulp.lpSum(self.x[j] for j in self.J)

        #constraints
        #1 and 2
        for i in self.I:
            for j in self.J:
                if i != j:
                    self.problem += self.x[j] + self.p[i] - self.x[i] <= self.M*self.z[(i,j)]
                    self.problem += self.x[i] + self.p[j] -self.x[j] <= self.M*(1 - self.z[(i,j)])
        #3
        for j in self.J:
            self.problem += self.x[j] >= self.p[j]
    
    def solve(self):
        self.problem.solve()
        if self.problem.status == pulp.LpStatusOptimal:
            print(f"Optimal value is: {pulp.value(self.problem.objective)}")
            for j in self.J:
                print(f"Job {j} completion time: {self.x[j].value()}")
            for i,j in self.z:
                if i != j:
                    print(f"z_{i}_{j} (Job {i} precedes Job {j}): {self.z[(i, j)].value()}")

        else:
            print("No optimal solution found.")

In [6]:
p = [3, 5, 2, 4]  # Processing times for jobs 1, 2, 3, and 4 respectively
I = J = [0, 1, 2, 3]  # Job indices in Python typically start at 0

single_machine = SingleMachineTCT(I,J,p)
single_machine.model()
single_machine.solve()

Optimal value is: 30.0
Job 0 completion time: 5.0
Job 1 completion time: 14.0
Job 2 completion time: 2.0
Job 3 completion time: 9.0
z_0_1 (Job 0 precedes Job 1): 1.0
z_0_2 (Job 0 precedes Job 2): 0.0
z_0_3 (Job 0 precedes Job 3): 1.0
z_1_0 (Job 1 precedes Job 0): 0.0
z_1_2 (Job 1 precedes Job 2): 0.0
z_1_3 (Job 1 precedes Job 3): 0.0
z_2_0 (Job 2 precedes Job 0): 1.0
z_2_1 (Job 2 precedes Job 1): 1.0
z_2_3 (Job 2 precedes Job 3): 1.0
z_3_0 (Job 3 precedes Job 0): 0.0
z_3_1 (Job 3 precedes Job 1): 1.0
z_3_2 (Job 3 precedes Job 2): 0.0
